In [1]:
import torchtext
from torchtext import data
from torchtext.data import Field
import pandas as pd
import numpy as np
import spacy
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,wordpunct_tokenize
import string
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

In [2]:
nlp = spacy.load('en')

In [3]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED)

In [4]:
stopwords = set(stopwords.words('english'))

In [5]:
stopwords.add("'s")
stopwords.add("...")

In [6]:
def text_preprocessing(x):
    x = str(x)
    
    #cleaning url, @name, emojis, smiley etc
    text = p.clean(x)
    
    #punctation removal
    no_punct = [word for word in word_tokenize(text) if word not in string.punctuation]
    
    #stopword removal
    no_sw = set(wrd for wrd in no_punct if wrd not in stopwords)
    
    return " ".join(w for w in no_sw if w).strip()

### Creating Dataset using TabularDataset from Torchtext

In [8]:
TEXT = Field(sequential=True, preprocessing = data.Pipeline(lambda x : text_preprocessing(x)),
            include_lengths=True)

# TEXT = Field(tokenize = 'spacy',
#              include_lengths=True)

#include_lengths - converts TEXT into a tuple of two elements, 
#one element is the numericalised tensor with pads
#other element is the actual length of our sentence

LABEL = data.LabelField(dtype = torch.float)

tweet = data.TabularDataset(path='train_original.csv', format='csv',
                            fields={
                                'text': ('text', TEXT),
                                'target': ('labels', LABEL)
                                   }
                           )

In [9]:
TEXT.build_vocab(tweet,max_size = 15000,
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(tweet)

In [10]:
#TEXT.build_vocab(tweet, max_size=10000)

In [11]:
#LABEL.build_vocab(tweet)

In [12]:
LABEL.vocab.freqs

Counter({'1': 3271, '0': 4342})

In [16]:
TEXT.vocab.freqs.pop('') # remove empty string

38587

In [17]:
train, valid = tweet.split(split_ratio=0.8)

### Convert Dataset into Iterator 

In [18]:
device = torch.device('cuda')

### BucketIterator - 

>**Shuffles the data in iterator**

>**It is recommended that for test data we should use just itertor, since we are evaluating the model.**

In [19]:
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, valid, None),batch_sizes=(64, 128, 0),
                                                            sort_key=lambda x: len(x.text), device=device)

In [20]:
type(train_iter.data()[0].text)

list

In [21]:
type(val_iter.data()[99].text)

list

In [22]:
test = pd.read_csv("data/test.csv")

In [23]:
%%time
test = data.TabularDataset(path='data/test.csv', format='csv',
                            fields={
                                'text': ('text', TEXT),
                                }
                           )

CPU times: user 2.92 s, sys: 0 ns, total: 2.92 s
Wall time: 2.92 s


In [24]:
test_iter = data.Iterator(dataset=test, device=device, batch_size=128, sort=False)

## Model Building

In [25]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [26]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [27]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,387,817 trainable parameters


In [28]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([15002, 100])


In [29]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.6093,  1.1288, -0.3286,  ...,  0.4432, -0.9317, -0.3256],
        [ 0.4628, -0.9749,  0.4209,  ..., -1.1212,  1.2555,  1.1448],
        [ 0.7467,  1.5334, -0.6696,  ...,  0.2919,  0.2122, -0.6511],
        ...,
        [ 0.2251,  0.5239, -1.4983,  ...,  2.3330, -0.2560, -0.5038],
        [-0.8306,  0.0777, -0.0441,  ..., -0.6532,  0.7827, -0.4262],
        [ 0.3743, -1.5479, -0.4312,  ..., -0.8107, -0.3086, -0.6888]])

In [30]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7467,  1.5334, -0.6696,  ...,  0.2919,  0.2122, -0.6511],
        ...,
        [ 0.2251,  0.5239, -1.4983,  ...,  2.3330, -0.2560, -0.5038],
        [-0.8306,  0.0777, -0.0441,  ..., -0.6532,  0.7827, -0.4262],
        [ 0.3743, -1.5479, -0.4312,  ..., -0.8107, -0.3086, -0.6888]])


In [31]:
TEXT.vocab.stoi[TEXT.unk_token]

0

In [32]:
print(model.embedding.weight.data.shape)

torch.Size([15002, 100])


In [33]:
optimizer = optim.Adam(model.parameters())

In [34]:
criterion = nn.BCEWithLogitsLoss()

In [35]:
model = model.to(device)
criterion = criterion.to(device)

In [36]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [37]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.labels)
        
        acc = binary_accuracy(predictions, batch.labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [38]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.labels)
            
            acc = binary_accuracy(predictions, batch.labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [39]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
N_EPOCHS = 25

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.620 | Train Acc: 66.46%
	 Val. Loss: 0.552 |  Val. Acc: 72.27%
Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.538 | Train Acc: 73.57%
	 Val. Loss: 0.548 |  Val. Acc: 73.04%
Epoch: 03 | Epoch Time: 0m 2s
	Train Loss: 0.497 | Train Acc: 76.78%
	 Val. Loss: 0.519 |  Val. Acc: 74.52%
Epoch: 04 | Epoch Time: 0m 2s
	Train Loss: 0.450 | Train Acc: 79.70%
	 Val. Loss: 0.495 |  Val. Acc: 75.84%
Epoch: 05 | Epoch Time: 0m 2s
	Train Loss: 0.426 | Train Acc: 81.44%
	 Val. Loss: 0.498 |  Val. Acc: 77.01%
Epoch: 06 | Epoch Time: 0m 2s
	Train Loss: 0.392 | Train Acc: 82.51%
	 Val. Loss: 0.508 |  Val. Acc: 76.84%
Epoch: 07 | Epoch Time: 0m 2s
	Train Loss: 0.362 | Train Acc: 84.76%
	 Val. Loss: 0.513 |  Val. Acc: 76.97%
Epoch: 08 | Epoch Time: 0m 2s
	Train Loss: 0.332 | Train Acc: 85.99%
	 Val. Loss: 0.574 |  Val. Acc: 76.90%
Epoch: 09 | Epoch Time: 0m 2s
	Train Loss: 0.316 | Train Acc: 87.15%
	 Val. Loss: 0.534 |  Val. Acc: 76.88%
Epoch: 10 | Epoch Time: 0m 2

In [50]:
test_preds = []

with torch.no_grad():
    for batch in test_iter:
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        preds = predictions.data.cpu().numpy()
        #preds = preds.data.numpy()
        # the actual outputs of the model are logits, so we need to pass these values to the sigmoid function
        preds = 1 / (1 + np.exp(-preds))
        test_preds.append(preds)

In [58]:
predictions = np.where(np.concatenate(test_preds, axis=0) > 0.5, 1, 0)

In [59]:
test_data = pd.read_csv('data/test.csv')
test_data["target"] = predictions
submission_file = test_data[['id', 'target']]
submission_file.to_csv("Submission_file.csv", index=False)